In [3]:
import json
import argparse
import subprocess
import boto3
import time
import pandas as pd
from pyathena import connect
import pandas.io.sql as sqlio
import sys
from ydata_profiling import ProfileReport
import numpy as np
from botocore import UNSIGNED
from botocore.config import Config
import boto3.session
from botocore import exceptions


/home/gabriel/anaconda3/envs/.conda/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/home/gabriel/.local/lib/python3.10/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [4]:
class CustomException(Exception):
    pass

json_manifest_dbt = "target/manifest.json"
athena_bucket = "todos-athena-us-east-1"
athena_tmp_folder = f"s3://{athena_bucket}/"

In [5]:
def execute_athena_sql(query):
    client = boto3.client('athena', region_name='us-east-1')
    queryStart = client.start_query_execution(
    QueryString = query,
    ResultConfiguration = { 'OutputLocation': athena_tmp_folder})
    queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])
    while queryExecution['QueryExecution']['Status']['State'] in ('RUNNING', 'QUEUED'):
        time.sleep(5)
        queryExecution = client.get_query_execution(QueryExecutionId=queryStart['QueryExecutionId'])

In [6]:
def execute_athena_query(query):
    cursor = connect(s3_staging_dir=athena_tmp_folder,
                    region_name="us-east-1").cursor()
    cursor.execute(query)
    colls=','.join(str(f"{e[0]}") for e in cursor.description)
    results=pd.DataFrame(list(cursor), columns=colls.split(","))
    return results

In [7]:
#Query dentro da PDGT
fl_especialidades = 'select * from pdgt_sandbox_gabrielguilherme.fl_especialidades'
fl_fornecedores ='select * from pdgt_sandbox_gabrielguilherme.fl_fornecedores'
fl_profissionais = 'select fp.id_profissional, fp.nm_profissional, fp.id_conselho, fp.nro_conselho, fp.cpf, fp.genero, fp.id_unidade, fp.unidade, fp.email1 , fp.email2, fp.telefone1, fp.telefone2, fp.celular1, fp.celular2, fp.dt_atualizacao, fp.dt_criacao, fp.status_cadastro from pdgt_sandbox_gabrielguilherme.fl_profissionais fp'
fl_regionais = 'select * from pdgt_sandbox_gabrielguilherme.fl_regionais'
fl_unidades = 'select * from pdgt_sandbox_gabrielguilherme.fl_unidades'
fl_agendamentos = 'select a.id_agendamento, a.id_regional, a.regional, a.id_unidade, a.unidade,  a.id_paciente, a.nm_paciente, a.cpf, a.celular, a.email, a.sexo, a.estado,a.cidade ,a.bairro ,a.logradouro ,a.numero ,a.id_status ,a.nm_status ,a.id_profissional ,a.nm_profissional ,a.id_especialidade ,a.nm_especialidade ,a.id_procedimento ,a.nm_procedimento ,a.id_tipoprocedimento ,a.id_grupoprocedimento ,a.id_tabela ,a.nm_tabela ,a.valor  from pdgt_sandbox_gabrielguilherme.fl_agendamentos a limit 10'
fl_canais = 'select * from pdgt_sandbox_gabrielguilherme.fl_canais'
fl_indicadores = 'select * from pdgt_sandbox_gabrielguilherme.fl_indicadores limit 100000000'
fl_pacientes = '''select * from todos_data_lake_trusted_feegow.pacientes p
where sysdate between date('2023-01-01') and date ('2023-12-31')
and p.sys_active = 1'''


In [8]:
df = '''with stg_agendamentos as (
select 'todos_data_lake_trusted_feegow.agendamento' as Tipo, cast(ag."data" as date), count(*) as Registros, 0 as valortotal, su.id as unidade_id
from todos_data_lake_trusted_feegow.agendamentos ag
left join todos_data_lake_trusted_feegow.locais sl on sl.id = ag.local_id
left join todos_data_lake_trusted_feegow.unidades su on su.id = sl.unidade_id
group by cast(ag."data" as date), su.id),
stg_movimentacao as (
select 'todos_data_lake_trusted_feegow.movimentacao' as Tipo, cast(ag."data" as date), count(*) as Registros, 0 as valortotal, ag.unidade_id 
	from todos_data_lake_trusted_feegow.movimentacao ag
group by cast(ag."data" as date), ag.unidade_id),
stg_contas as (
select 'todos_data_lake_trusted_feegow.contas' as Tipo ,cast(ag.data_referencia as date) as data, count(*) as Registros, 0 as valortotal, ag.unidade_id
	from todos_data_lake_trusted_feegow.contas ag
group by cast(ag.data_referencia as date), ag.unidade_id),
stg_propostas as (
select 'todos_data_lake_trusted_feegow.propostas' as Tipo, cast(p.dataproposta as date) as data, count(*) as Registros, 0 as valortotal, p.unidadeid as unidade_id
from todos_data_lake_trusted_feegow.propostas p
group by cast(p.dataproposta as date),p.unidadeid),
stg_memed_prescricoes as (
select 'todos_data_lake_trusted_feegow.memed_prescricoes mp' as Tipo, cast(mp.datahora as date) as data  , count(*) as Registros, 0 as valortotal, sa.unidade_id
from todos_data_lake_trusted_feegow.memed_prescricoes mp
left join todos_data_lake_trusted_feegow.atendimentos sa on sa.id = mp.atendimentoid 
group by cast(mp.datahora as date), sa.unidade_id),
stg_dc_pdf_assinados as (
select 'todos_data_lake_trusted_feegow.dc_pdf_assinados' as Tipo, cast(pa.data_criacao as date) as data, count(*) as Registros, 0 as valortotal, ah.unidade_id 
from todos_data_lake_trusted_feegow.dc_pdf_assinados pa
left join todos_data_lake_trusted_feegow.atendimentos ah on ah.id = pa.documento_id 
group by cast(pa.data_criacao as date), ah.unidade_id),
stg_pacientes_pedido as (
select 'todos_data_lake_trusted_feegow.pacientes_pedidos' as Tipo, cast(pd."data"  as date) as data, count(*) as Registros, 0 as valortotal, sah.unidade_id
from todos_data_lake_trusted_feegow.pacientes_pedidos pd
left join todos_data_lake_trusted_feegow.atendimentos sah on sah.paciente_id  = pd.paciente_id  
group by cast(pd."data"  as date),sah.unidade_id),
stg_pacientes_prescricoes as (
select 'todos_data_lake_trusted_feegow.pacientes_prescricoes' as Tipo, cast(pp."data" as date) as data, count(*) as Registros, 0 as valortotal, sah.unidade_id 
from todos_data_lake_trusted_feegow.pacientes_prescricoes pp
left join todos_data_lake_trusted_feegow.atendimentos sah on sah.paciente_id = pp.paciente_id
group by cast(pp."data" as date), sah.unidade_id),
stg_agenda_horarios_itens as (
select 'todos_data_lake_trusted_feegow.agenda_horarios_itens' as Tipo ,cast(hi."data" as date) as data, count(*) as Registros, 0 as valortotal, hi.unidade_id 
from todos_data_lake_trusted_feegow.agenda_horarios_itens hi
group by cast(hi."data" as date), hi.unidade_id),
stg_contas_bloqueios as (
select 'todos_data_lake_trusted_feegow.contas_bloqueios' as Tipo, cast(cb."data" as date)as data , count(*) as Registros, 0 as valortotal, cb.unidade_id 
from todos_data_lake_trusted_feegow.contas_bloqueios cb
group by cast(cb."data" as date), cb.unidade_id),
stg_atendimentos as (
select 'todos_data_lake_trusted_feegow.atendimentos' as Tipo, cast(a."data" as date) as data, count(*) as Registros, 0 as valortotal, a.unidade_id 
from todos_data_lake_trusted_feegow.atendimentos a
group by cast(a."data" as date), a.unidade_id),
stg_agendamento_procedimentos as (
select 'todos_data_lake_trusted_feegow.agendamento_procedimentos sap' as Tipo, cast(sap.dhup as date) as data, count(*) as Registros, 0 as valortotal, su.id as unidade_id
from todos_data_lake_trusted_feegow.agendamento_procedimentos sap 
left join todos_data_lake_trusted_feegow.locais sl on sl.id = sap.local_id
left join todos_data_lake_trusted_feegow.unidades su on su.id = sl.unidade_id 
group by cast(sap.dhup as date), su.id),
stg_procedimentos as (
select 'todos_data_lake_trusted_feegow.procedimentos' as Tipo, cast(sp.dhup as date) as data ,count(*) as Registros, 0 as valortotal, su.id as unidade_id
from todos_data_lake_trusted_feegow.procedimentos sp
left join todos_data_lake_trusted_feegow.agendamentos sah on sah.procedimento_id = sp.id
left join todos_data_lake_trusted_feegow.locais sl on sl.id = sah.local_id 
left join todos_data_lake_trusted_feegow.unidades su on su.id = sl.unidade_id 
group by cast(sp.dhup as date), su.id),
tb_consolidacao_agendamentos_hist as (
select 'tb_consolidacao_agendamentos_hist' as Tipo, cast(ahi.dt_agendamento as date) as data, count(*) as Registro, 0 as valortotal, ahi.id_unidade as unidade_id 
from pdgt_amorsaude_operacoes.fl_agendamentos ahi
group by cast(ahi.dt_agendamento as date), ahi.id_unidade),
tb_consolidacao_contas_a_receber as (
select 'tb_consolidacao_contas_a_receber_hi' as Tipo, cast(cr.datavencimento  as date) as data, count(*) as Registros, cr.valor_pago as valortotal, cr.id_unidade  as unidade_id
from pdgt_amorsaude_financeiro.fl_contas_a_receber cr
group by cast(cr.datavencimento as date), cr.valor_pago, cr.id_unidade), --problema em varchar dentro do contas a receber
tb_consolidacao_receita_bruta_hist_final as (
select 'tb_consolidacao_receita_bruta_hist_final' as Tipo, cast(tcrbhf."data" as date) as data, count(*) as Registros, tcrbhf.total_recebido as valortotal, tcrbhf .id_unidade  as unidade_idc
from pdgt_amorsaude_financeiro.fl_receita_bruta tcrbhf
group by cast(tcrbhf."data" as date),tcrbhf.total_recebido, tcrbhf .id_unidade),
tb_consolidacao_contas_a_pagar_hist as (
select 'tb_consolidacao_contas_a_pagar_hist' as Tipo, cast(cp."data" as date) as data, count(*) as Registros, cp.valortotal as valortotal, su.id as unidade_id
from pdgt_amorsaude_financeiro.fl_contas_a_pagar cp
left join todos_data_lake_trusted_feegow.unidades su on su.nome_fantasia = cp.nm_unidade 
group by cast(cp."data" as date), cp.valortotal, su.id),
stg_boletos_emitidos as (
select 'todos_data_lake_trusted_feegow.boletos_emitidos' as Tipo, cast(be.data_hora as date), count(*) as Registros, 0 as valortotal, be.unidade_id 
from todos_data_lake_trusted_feegow.boletos_emitidos be
group by cast(be.data_hora as date), be.unidade_id),
stg_grade_fixa as (
select 'todos_data_lake_trusted_feegow.grade_fixa' as Tipo, cast(gf.datahora as date), count(*) as Registros, 0 as valortotal, su.id as unidade_id
from todos_data_lake_trusted_feegow.grade_fixa gf
left join todos_data_lake_trusted_feegow.locais sl on sl.id = gf.localid
left join todos_data_lake_trusted_feegow.unidades su on su.id = sl.unidade_id 
group by cast(gf.datahora as date), su.id),
stg_royalties_contas as (
select 'todos_data_lake_trusted_feegow.grade_fixa' as Tipo, cast(rc.datareferencia as date), count(*) as Registros, 0 as valortotal, rc.unidadeid as unidade_id
from todos_data_lake_trusted_feegow.royalties_contas rc
group by cast(rc.datareferencia as date), rc.unidadeid),
stg_splits as (
select 'todos_data_lake_trusted_feegow.splits' as Tipo, cast(s."data" as date), count(*) as Registros, 0 as valortotal, smh.unidade_id
from todos_data_lake_trusted_feegow.splits s
left join todos_data_lake_trusted_feegow.movimentacao smh on smh.id = s.movimentacao_id 
group by cast(s."data" as date), smh.unidade_id),
stg_conta_itens as (
select 'todos_data_lake_trusted_feegow.conta_itens' as Tipo, cast(sci.data_execucao as date) as data, count(*) as Registros, 0 as valortotal, 0 as unidade_id
from todos_data_lake_trusted_feegow.conta_itens sci
--left join todos_data_lake_trusted_feegow.contas scc on scc.conta_id = sci.id
group by cast(sci.data_execucao as date))
select * from (
select * from stg_conta_itens
union all
select * from stg_contas sc 
union all
select * from stg_splits ss 
union all
select * from stg_propostas
union all
select * from stg_grade_fixa
union all
select * from stg_grade_fixa
union all
select * from stg_boletos_emitidos
union all
select * from stg_procedimentos
union all
select * from stg_agendamento_procedimentos
union all
select * from stg_atendimentos
union all
select * from stg_dc_pdf_assinados
union all
select * from stg_pacientes_pedido
union all
select * from stg_pacientes_prescricoes
union all
select * from stg_agenda_horarios_itens
union all
select * from stg_contas_bloqueios
union all
select * from stg_memed_prescricoes
union all
select * from stg_agendamentos
union all
select * from stg_movimentacao
union all
select * from tb_consolidacao_agendamentos_hist
union all
select * from tb_consolidacao_contas_a_receber
union all
select * from tb_consolidacao_receita_bruta_hist_final
union all
select * from tb_consolidacao_contas_a_pagar_hist
WHERE data BETWEEN date_add('day', -90, current_date) AND current_date
and data <= current_date
order by data desc
)'''

In [9]:
df = execute_athena_query(fl_pacientes)

In [10]:
#profile = ProfileReport(df, lazy=True, dark_mode=True,  title="Data Profiling AmorSaúde")
#profile.to_notebook_iframe()

In [11]:
pip install sweetviz

Note: you may need to restart the kernel to use updated packages.


In [13]:
import sweetviz as sv

report = sv.analyze(df)

# Salvar o relatório em HTML
report.show_html("data_profile_report.html")

                                             |          | [  0%]   00:00 -> (? left)

: 

In [ ]:
# Convertendo a coluna 'data' para o tipo datetime
df['data'] = pd.to_datetime(df['data'])

# Simulando uma nova query com dados a partir de uma data específica
nova_query = pd.DataFrame({
    'Cliente': ['D', 'E'],
    'Valor': [250, 300],
    'DataPagamento': ['2022-01-03', '2022-01-04']
})

# Convertendo a coluna 'data' para o tipo datetime
nova_query['data'] = pd.to_datetime(nova_query['data'])

# Data a partir da qual você deseja atualizar incrementalmente
data_atualizacao = pd.to_datetime('today') - pd.DateOffset(1)

# Filtrando os novos dados com base na condição de data
novos_dados = nova_query[nova_query['data'] == data_atualizacao]

# Atualizando incrementalmente o DataFrame existente
df_atualizado = pd.concat([df, novos_dados], ignore_index=True)

# Exibindo o DataFrame atualizado
df_atualizado

In [ ]:
profile.to_file("your_report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]